In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os

In [ ]:
thedir = '/content/drive/MyDrive/Colab Notebooks/food recognition/images/train'
categories = [ name for name in os.listdir(thedir) if os.path.isdir(os.path.join(thedir, name)) ]

In [ ]:
print(categories)

['club_sandwich', 'hot_dog', 'churros', 'cheese_plate', 'donuts', 'sashimi', 'pad_thai', 'cheesecake', 'crab_cakes', 'lobster_bisque', 'caprese_salad', 'gyoza', 'beef_tartare', 'french_fries', 'pork_chop', 'bread_pudding', 'breakfast_burrito', 'onion_rings', 'fish_and_chips', 'ice_cream', 'miso_soup', 'deviled_eggs', 'chocolate_cake', 'pho', 'tuna_tartare', 'foie_gras', 'grilled_cheese_sandwich', 'cup_cakes', 'ceviche', 'chocolate_mousse', 'greek_salad', 'hamburger', 'caesar_salad', 'eggs_benedict', 'pancakes', 'guacamole', 'creme_brulee', 'edamame', 'clam_chowder', 'risotto', 'shrimp_and_grits', 'filet_mignon', 'nachos', 'fried_rice', 'ravioli', 'peking_duck', 'pizza', 'macarons', 'bibimbap', 'mussels', 'french_toast', 'french_onion_soup', 'garlic_bread', 'beet_salad', 'escargots', 'dumplings', 'baklava', 'croque_madame', 'spaghetti_bolognese', 'sushi', 'grilled_salmon', 'omelette', 'tacos', 'baby_back_ribs', 'waffles', 'macaroni_and_cheese', 'hummus', 'poutine', 'gnocchi', 'ramen', '

In [ ]:
image_size = [224, 224]

train = '/content/drive/MyDrive/Colab Notebooks/food recognition/images/train'
test = '/content/drive/MyDrive/Colab Notebooks/food recognition/images/test'

In [ ]:
vgg = VGG19(input_shape=image_size + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
folders = glob('/content/drive/MyDrive/Colab Notebooks/food recognition/images/train/*')
len(folders)

101

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train, 
                                                 target_size = [224, 224], 
                                                 batch_size = 32, 
                                                 class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test, 
                                            target_size = [224, 224], 
                                            batch_size = 32, 
                                            class_mode = 'categorical')

Found 1010 images belonging to 101 classes.
Found 416 images belonging to 101 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/20
32/32 [==============================] - 19s 565ms/step - loss: 7.5349 - accuracy: 0.0149 - val_loss: 5.7644 - val_accuracy: 0.0721
Epoch 2/20
32/32 [==============================] - 18s 556ms/step - loss: 3.2884 - accuracy: 0.3356 - val_loss: 5.0290 - val_accuracy: 0.1082
Epoch 3/20
32/32 [==============================] - 18s 567ms/step - loss: 2.0197 - accuracy: 0.5495 - val_loss: 5.3343 - val_accuracy: 0.1514
Epoch 4/20
32/32 [==============================] - 18s 559ms/step - loss: 1.2257 - accuracy: 0.7287 - val_loss: 5.0144 - val_accuracy: 0.1346
Epoch 5/20
32/32 [==============================] - 18s 571ms/step - loss: 0.7623 - accuracy: 0.8208 - val_loss: 5.0021 - val_accuracy: 0.1562
Epoch 6/20
32/32 [==============================] - 18s 563ms/step - loss: 0.5143 - accuracy: 0.8911 - val_loss: 5.0542 - val_accuracy: 0.1538
Epoch 7/20
32/32 [==============================] - 18s 554ms/step - loss: 0.3022 - accuracy: 0.9277 - val_loss: 4.9769 - val_accuracy: 0.1683

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/Colab Notebooks/food recognition/model_food_recognition_vgg19.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
m=load_model('/content/drive/MyDrive/Colab Notebooks/food recognition/model_food_recognition_vgg19.h5')

In [ ]:
img=image.load_img('/content/drive/MyDrive/Colab Notebooks/food recognition/index.jpeg',target_size=(224,224))

In [ ]:
x = image.img_to_array(img)


In [ ]:
x=x/255
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
y_out = np.argmax(model.predict(img_data), axis=1)
y_out = categories[y_out[0]]
print(y_out)

crab_cakes
